## Optimization TD1 - Notebook

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from div import div
from grad import grad

In [ ]:
def load_image(filename):
    return np.loadtxt(filename)

def display_image(image, title="Image"):
    plt.figure()
    plt.imshow(image, cmap='gray')
    plt.title(title)
    plt.savefig(f"outputs/{title}.png")
    plt.show()

In [ ]:
# Question 1

# load image
z = load_image("marie_degraded")

# image size
K, L = z.shape
print(f"The image size is {K} x {L}")

# display image
display_image(z, "Marie Degraded")

In [ ]:
# Question 2

# indices corresponding to tearing
tear_mask = (z == 0)
indJ = np.where(tear_mask)
print(f"Tearing represents {100 * len(indJ[0]) / (K * L)} % of the image")

# indices of complementary area    
indI = np.where(~tear_mask)
print(f"Complementary represents {100 * len(indI[0]) / (K * L)} % of the image")

In [ ]:
def gradient_algorithm(z, nitm, gamma):
    K, L = z.shape
    xr = z.copy()
    cost = []
    for nit in range(nitm):
        v, h = grad(xr)
        xr = xr - gamma * div(v, h)
        cost.append((np.linalg.norm(v, 'fro')**2 + np.linalg.norm(h, 'fro')**2) / 2)
        print(f"{nit+1} : cost={cost[-1]}")
    return xr, cost

In [ ]:
# Question 4
# gradient algorithm for minimizing g o L

nitm = 15000 # maximum number of iterations
beta = 8 # Lipshitz constant of the gradient
gamma = 1.9 / beta # step-size of the algorithm
xr, cost = gradient_algorithm(z, nitm, gamma)
display_image(xr, "Restored Image with Quadratic Cost")

In [ ]:
def projball2(xr, z, rho, indI):
    """
    Projection of xr onto the L2 ball || x(indI) - z(indI) || <= rho.

    Parameters:
    xr : ndarray (vector to be projected)
    z : ndarray (center of the ball)
    rho : float (radius of the ball)
    indI : ndarray (indices of constrained components)

    Returns:
    p : ndarray (projected vector)
    """
    p = xr.copy()
    no = np.linalg.norm(xr[indI] - z[indI])

    if no > rho:
        p[indI] = z[indI] + rho * (xr[indI] - z[indI]) / no

    return p

In [ ]:
def projected_gradient_algorithm(z, nitm, rho, gamma, indI, prec=1e-7):
    xr = z.copy()
    cost = []
    
    for nit in range(nitm):
        v, h = grad(xr)
        xr = xr - gamma * div(v, h)
        xr = projball2(xr, z, rho, indI)

        cost.append((np.linalg.norm(v, 'fro')**2 + np.linalg.norm(h, 'fro')**2) / 2)
        print(f"{nit+1} : cost={cost[-1]}")

        if nit > 0 and abs((cost[-2] - cost[-1]) / cost[-1]) < prec:
            break

    return xr, cost

In [ ]:
# Questions 6-8
# projected gradient algorithm for minimizing g o L subject to constraint
nitm = 15000 # maximum number of iterations
rho = 0.2 * np.sqrt(K * L)
precc = 1e-7; # precision for stopping criterion
xr, cost = projected_gradient_algorithm(z, nitm, rho, gamma, indI, precc)
display_image(xr, "Restored Image with Constraint")
plt.plot(cost)
plt.title("Convergence Plot")
plt.savefig("outputs/ConvergencePlot.png")
plt.show()